In [1]:
import sympy as sp
import numpy as np
from model import S, R, T, to_line, tan_simplify, Maclaurin_series, simplify_array, Taylor_series, truncate, diff_on_S1, jacobian
import model as m
from sympy import sqrt, acos, atan, symbols
from numpy.linalg import norm, inv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
plt.ion()

In [2]:
r , constraints = m.get_leg_position()
Z = sp.symbols('z_a, z_d, z_y')
M = sp.symbols(r'\alpha_h, \alpha_r, \alpha_b')
z_a, z_d, z_y = Z
Beta = m.beta_A, m.beta_D, m.beta_Y
U = m.u_h, m.u_r, m.u_b
a_h, a_r, a_b = M
subs = []

for z, u in zip(M, U):
    subs.extend([(sp.sin(u), (z - 1/z)/2j),  (sp.cos(u), (z + 1/z)/2)])
    
for z, beta in zip(Z,Beta):
    subs.extend([(sp.sin(beta), (z - 1/z)/2j),  (sp.cos(beta), (z + 1/z)/2)])
        
r = [sp.simplify(r_i.subs(subs)) for r_i in r]
constraints = [sp.simplify(sp.expand_trig(c).subs(subs)) for c in constraints]
#r[0] = sp.expand(r[0]* z_a * z_d * z_y)
#r[1] = sp.expand(r[1]*z_a*z_d*z_d*z_y)
#r[2]= sp.expand(r[2]*z_y*z_d)
constraints[0] = truncate(sp.expand(constraints[0]))
constraints[1] = sp.expand(constraints[1]*a_h*z_d)
constraints[2] = sp.expand(constraints[2]*z_y*a_r)

#
# want d/dt 
# 
F, w = jacobian(r)

In [3]:
r , constraints = m.get_leg_position()
sp.expand_trig(constraints[0])

-3.35731442646649e-13*sin(\beta_A)**2 + 900.0*sin(\beta_A)*sin(u_b) - 400.0*sin(\beta_A)*cos(u_b) - 3024.0*sin(\beta_A) + 400.0*sin(u_b)*cos(\beta_A) - 1050.0*sin(u_b) + 900.0*cos(\beta_A)*cos(u_b) - 1344.0*cos(\beta_A) + 1872.25

In [4]:
G, w_ref = jacobian(constraints)
dG_z = G[0:3, ::2]
dG_a = G[0:3, 1::2]


In [5]:
F

Matrix([
[                                                                                   -I*(-1.0*z_a**2*z_d*z_y*(16.0*z_d + 7.5*z_y - 37.5*I*z_y + 33.0) + 33.0*z_d*z_y + 1.0*z_d*(7.5 + 37.5*I) + 16.0*z_y)/(z_a*z_d*z_y),                                                                             I*(z_a**2*z_y**2*(7.5 - 37.5*I) - 7.5 - 37.5*I)/(z_a*z_y),                       16.0*I*z_a*z_d - 16.0*I/(z_a*z_d)],
[8.0*z_a*z_d + 3.75*z_a*z_y - 18.75*I*z_a*z_y + 33.0*z_a + 3.75*z_a/z_y + 18.75*I*z_a/z_y + 8.0*z_a/z_d + 8.0*z_d/z_a + 3.75*z_y/z_a - 18.75*I*z_y/z_a + 33.0/z_a + 3.75/(z_a*z_y) + 18.75*I/(z_a*z_y) + 8.0/(z_a*z_d), 3.75*z_a*z_y - 18.75*I*z_a*z_y - 3.75*z_a/z_y - 18.75*I*z_a/z_y - 3.75*z_y/z_a + 18.75*I*z_y/z_a + 3.75/(z_a*z_y) + 18.75*I/(z_a*z_y), 8.0*z_a*z_d - 8.0*z_a/z_d - 8.0*z_d/z_a + 8.0/(z_a*z_d)],
[                                                                                                                                                                            

### Goal
In the body frame, we have the $i$ th leg 
$$ (x,y,z)_i^T = f_i(\theta^i_t, \theta^i_m, \theta^i_b)$$
we want to choose a function $\theta^i_t, \theta^i_m, \theta^i_b = \gamma(s)$ such that $s \in [0, 1]$

Each leg consists of two 4R linkages and a parallelogram.

Two key pivot points are given by the horizontal plane of the D brace.

The two RRRR linkages define two derived angles.

In [6]:
beta_D, beta_Y = sp.symbols(r'\beta_D, \beta_y', real=True)
u_r, u_h = sp.symbols('u_r, u_h', real=True)

In [7]:
Leg_Pivot = R(beta_D) @ T(m.D_section_e, 0) @ R(-beta_D)
Leg_Bottom = Leg_Pivot @ R(beta_Y) @ T(m.leg_pivot_to_foot_x, m.leg_pivot_to_foot_y)

In [8]:
# For R(beta_Y)
Y_inner = R(beta_Y) @ T(m.Y_i,m.Y_h) @ R(-beta_Y)
Y_outer = R(beta_Y) @ T(m.Y_o,m.Y_h) @ R(-beta_Y)
D_bottom = R(beta_D) @ T(0, m.D_section_b) @ R(-beta_D) 
Top_Motor = T(m.motor_offset_x, m.motor_offset_top)   
Top_Motor_Anchor = Top_Motor @ R(u_r) @ T(0, m.motor_arm_top) @ R(-u_r) 
Bottom_Motor = T(m.motor_offset_x, m.motor_offset_bottom)
Bottom_Motor_Anchor = Bottom_Motor @ R(u_h) @ T(0, m.motor_arm_bottom) @ R(-u_h)

In [9]:
## Bottom Linkage constraint
C_b = ((Bottom_Motor_Anchor - D_bottom).T) @ (Bottom_Motor_Anchor - D_bottom)
simplify_array(C_b)
f_b = sp.simplify(sp.expand(C_b[2,2] - m.l_b**2))
# f_b = f_b.subs(u_h, sp.asin(u_h))
F_b = sp.lambdify((u_h, beta_D),f_b )
theta = np.linspace(-np.pi/2, np.pi/2, 30)
interval = np.linspace(-1/2, 1/2, 30)
xz_i, yz_i = np.meshgrid(interval, theta)
plt.contour(xz_i, yz_i, F_b(xz_i,yz_i), [0])

<IPython.core.display.Javascript object>

In [10]:
eq = sp.collect(sp.expand_trig(f_b), beta_D)
f_beta_d = tan_simplify(eq, beta_D, implicit=False, positive_branch=True)
beta_d_series = (Maclaurin_series(f_beta_d, u_h, order=1) - sp.pi)
f_beta_d_series = sp.lambdify(u_h, beta_d_series)
plt.contour(xz_i,yz_i, F_b(xz_i,yz_i), [0])
plt.plot(theta, f_beta_d_series(theta), ':')

In [11]:
## Top Linkage Constraint
C_t = ((Top_Motor_Anchor - Y_inner).T )@  (Top_Motor_Anchor - Y_inner)
C_t = simplify_array(C_t)
        
f_t = sp.simplify(sp.expand(C_t[2,2] - m.l_d**2))
# f_t = sp.simplify(f_t.subs(u_r, sp.atan(u_r)))
F_t = sp.lambdify((u_r, beta_Y), f_t)

eq = sp.collect(sp.expand_trig(f_t), beta_Y)
f_beta_Y_i = tan_simplify(eq, beta_Y, implicit=True)
f_beta_Y = tan_simplify(eq, beta_Y, implicit=False)
beta_Y_series = Taylor_series(f_beta_Y, u_r, 0, order=1)
f_beta_Y_series = sp.lambdify(u_r, beta_Y_series)
xz, yz = np.meshgrid(theta, theta)
plt.contour(xz,yz, F_t(xz,yz), [0])
#plt.contour(xz,yz, sp.lambdify((u_r, beta_Y), sp.re(f_beta_Y_i))(xz,yz), [0], linestyles='dotted')
plt.plot(theta, f_beta_Y_series(theta), ':')
beta_Y_series

0.315522210263401*u_r - 0.112568880852128

In [12]:
subs = [(beta_Y, beta_Y_series), (beta_D, beta_d_series)] 

In [13]:
o = np.array([0,0,1]).T
p = Leg_Pivot @ o
mt = Top_Motor @ o
mb = Bottom_Motor @ o

#plt.plot(*to_line(Leg_Pivot @ m.o, m.o))
#plt.plot(*to_line(Leg_Bottom @ m.o, p))
#plt.plot(*to_line(Y_inner @ m.o, m.o))
#plt.plot(*to_line(Y_outer @ m.o, m.o))
#plt.plot(*to_line(Y_outer @ p, p))
#plt.plot(*to_line(D_bottom@ m.o, p))
#plt.plot(*to_line(D_bottom@ m.o, m.o))
#plt.plot(*to_line(Y_outer @ p, Y_outer @ m.o))
#plt.plot(*to_line(Y_outer @ p,Leg_Bottom @ m.o ))
#plt.plot(*to_line(Top_Motor @ m.o , m.o))
#plt.plot(*to_line(Bottom_Motor@m.o, m.o))
#plt.plot(*to_line(Bottom_Motor_Anchor@m.o, mb))
#plt.plot(*to_line(Top_Motor_Anchor @m.o, mt))

In [14]:
foot = simplify_array(Leg_Bottom) @o
foot[0] = tan_simplify(foot[0], beta_Y)
foot[1] = tan_simplify(foot[1], beta_Y)


In [15]:
foot_x = sp.lambdify((u_r, u_h), foot[0].subs(subs))
foot_y = sp.lambdify((u_r, u_h), foot[1].subs(subs))

In [16]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xz,yz, foot_x(xz,yz) - foot_x(0,0))

ax.set_xlabel('Top Motor Angle')
ax.set_ylabel('Bottom Motor Angle')

<IPython.core.display.Javascript object>

Text(0.5, 0, 'Bottom Motor Angle')

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xz,yz, foot_y(xz,yz) - foot_y(0,0))
a = ax.plot([0],[0], [0])

<IPython.core.display.Javascript object>

In [18]:
plt.figure()
plt.contour(xz,yz, foot_y(xz,yz) - foot_y(0,0), [0, 2, 5, 10, 15])

<IPython.core.display.Javascript object>

In [19]:
# Level Set 
# h =  foot_y(b_d, b_y) - foot_y(0, 0)
# which is h = −32.0sin(𝛽𝐷)−76.4852927038918cos(𝛽𝑦−0.197395559849881)+68.710755215083
beta_d_ref = -sp.asin(2.39016539699662*sp.cos(beta_Y - 0.197395559849881))
f_beta_ref = sp.lambdify(beta_Y, beta_d_ref) 


In [20]:
foot[1].diff(beta_D), foot[1].diff(beta_Y)

(-32*cos(\beta_D), 15*sqrt(26)*sin(\beta_y - atan(1/5)))

In [21]:
beta_d_series - beta_d_ref

0.497237151828558*u_h + asin(2.39016539699662*cos(\beta_y - 0.197395559849881)) - pi + 3.01218217722285

In [23]:
print(beta_d_ref)

-asin(2.39016539699662*cos(\beta_y - 0.197395559849881))


In [24]:
sp.expand_trig(f_b).subs(sp.cos(beta_D), beta_D).subs(sp.sin(beta_D), sp.sqrt(1 - beta_D**2))

-650.0*\beta_D*cos(u_h) - 338.0*\beta_D - 650.0*sqrt(1 - \beta_D**2)*sin(u_h) - 1664.0*sqrt(1 - \beta_D**2) + 800.0*sin(u_h) + 162.5*cos(u_h) + 602.5

In [25]:
sp.solveset(f_b.subs(u_h, 0))

Union(ImageSet(Lambda(_n, 2*_n*pi - 0.942211999553979 + pi), Integers), ImageSet(Lambda(_n, 2*_n*pi - 0.129410476366948 + 2*pi), Integers))

In [26]:
from scipy.optimize import root

In [27]:
f_b_lambda = lambda X: [abs(sp.lambdify((beta_D, u_h), f_b)(X[0], X[1])), X[1]]
root(f_b_lambda, x0=[0, 0]).x[0]

-0.12941047636694816

In [28]:
_

-0.12941047636694816

In [29]:
f_t_lambda =  lambda X: [abs(sp.lambdify((beta_Y, u_r), f_t)(X[0], X[1])), X[1]]
root(f_t_lambda, x0=[0, 0]).x[0]

-0.11256888085212823

In [30]:
f_b

-1664.0*sin(\beta_D) + 800.0*sin(u_h) - 338.0*cos(\beta_D) + 162.5*cos(u_h) - 650.0*cos(\beta_D - u_h) + 602.5

In [31]:
a[0].__dict__

{'_stale': True,
 'stale_callback': <function matplotlib.artist._stale_axes_callback(self, val)>,
 '_axes': <matplotlib.axes._subplots.Axes3DSubplot at 0x1dd69d93130>,
 'figure': <Figure size 640x480 with 1 Axes>,
 '_transform': <matplotlib.transforms.CompositeGenericTransform at 0x1dd69ddbaf0>,
 '_transformSet': True,
 '_visible': True,
 '_animated': False,
 '_alpha': None,
 'clipbox': <matplotlib.transforms.TransformedBbox at 0x1dd68dab5e0>,
 '_clippath': None,
 '_clipon': True,
 '_label': '_line0',
 '_picker': None,
 '_contains': None,
 '_rasterized': None,
 '_agg_filter': None,
 '_mouseover': False,
 'eventson': False,
 '_oid': 0,
 '_propobservers': {},
 '_remove_method': <function list.remove(value, /)>,
 '_url': None,
 '_gid': None,
 '_snap': None,
 '_sketch': None,
 '_path_effects': [],
 '_sticky_edges': _XYPair(x=[], y=[]),
 '_in_layout': True,
 '_dashcapstyle': 'butt',
 '_dashjoinstyle': 'round',
 '_solidjoinstyle': 'round',
 '_solidcapstyle': 'projecting',
 '_linestyles': Non

<function _lambdifygenerated(u_r, u_h)>

In [33]:
foot[0]

-32*cos(\beta_D) - 15*sqrt(26)*cos(\beta_y + atan(5))

In [34]:
foot = simplify_array(Leg_Bottom) @o

In [35]:
foot[0]

75*sin(\beta_y) - 32*cos(\beta_D) - 15*cos(\beta_y)

In [38]:
foot[1]

-32*sin(\beta_D) - 15*sin(\beta_y) - 75*cos(\beta_y)

-1664.0*sin(\beta_D) + 800.0*sin(u_h) - 338.0*cos(\beta_D) + 162.5*cos(u_h) - 650.0*cos(\beta_D - u_h) + 602.5